Explore TwitterAPI
====

Based on this code: http://geduldig.github.io/TwitterAPI/faulttolerance.html

In [21]:
reset -fs

In [32]:
from time import sleep

from credentials import credentials
from ftfy import fix_text
from textblob import TextBlob
from TwitterAPI import TwitterAPI

In [23]:
creds = credentials.require(['access_token', 
                             'access_token_secret', 
                             'consumer_key',
                             'consumer_secret'])

api = TwitterAPI(creds.consumer_key,
                 creds.consumer_secret,
                 creds.access_token,
                 creds.access_token_secret)

In [24]:
def tweets(query):
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q':query}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield(item['text'])
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [25]:
t = tweets(query='pizza')

In [34]:
for _ in range(10):
    text = fix_text(next(t))
    print(text)
    
    blob = TextBlob(text)
    print(blob.tags)
    
    print('#'*40)
    sleep(1)

RT @TheCarlosPena: I just want to cruise forever. All of you can come with me AND we can eat all the pizza we want!! Anyone in? https://t.c…
[('RT', 'NNP'), ('@', 'NNP'), ('TheCarlosPena', 'NNP'), ('I', 'PRP'), ('just', 'RB'), ('want', 'VBP'), ('to', 'TO'), ('cruise', 'VB'), ('forever', 'RB'), ('All', 'DT'), ('of', 'IN'), ('you', 'PRP'), ('can', 'MD'), ('come', 'VB'), ('with', 'IN'), ('me', 'PRP'), ('AND', 'IN'), ('we', 'PRP'), ('can', 'MD'), ('eat', 'VB'), ('all', 'PDT'), ('the', 'DT'), ('pizza', 'NN'), ('we', 'PRP'), ('want', 'VBP'), ('Anyone', 'NN'), ('in', 'IN'), ('https', 'NN'), ('//t.c…', 'NN')]
########################################
RT @TheCarlosPena: I just want pizza right now. #random
[('RT', 'NNP'), ('@', 'NNP'), ('TheCarlosPena', 'NNP'), ('I', 'PRP'), ('just', 'RB'), ('want', 'VBP'), ('pizza', 'JJ'), ('right', 'RB'), ('now', 'RB'), ('random', 'NN')]
########################################
Comiendo pizza a las 2:30 de la mañana jajaj https://t.co/A8i2xCF1R9
[('Comiendo', 

---
__You get a Rainbow Butterfly Unicorn Kitten:__

![](https://s-media-cache-ak0.pinimg.com/736x/bf/3f/4c/bf3f4c4e4cbc909f957f939bb6bc7cc6.jpg)